# TYNDP Translator

Converts TYNDP data into the LEGO format. The TYNDP models future power grid requirements by representing bidding zones and their respective transport capacities.

Figure 1 provides an overview of the bidding zone names.

<figure>
    <img src="assets/Bidding_Zones.png" alt="ENTSOE Bidding Zones" width="500"/>
    <figcaption>Figure 1: Bidding Zones according to ENTSO-E.</figcaption>
</figure>

---
## 1. Define Parameters

In [ ]:
## Paths
import os

# Simulation year (2030, 2040, 2050)
simulation_year = 2030

Datasource = 'TYNDP2024'  # Raw Data source

# Scenario (National Trends, Distributed Energy, Global Ambition)
scenario = 'Distributed Energy' 

# Climate year (1982-2019). The three years by TYNDP2024: 1995, 2008, 2009. Standard: 2009
cy_year = 2009

# Path to TYNDP input data (Backup Available at Institutsdaten\Daten\TYNDP2024)
tyndp_dir = 'L:\TYNDP 24\Data'

# Path to template LEGO files (by default stored with script on GitHub)
template_dir = 'L:\TYNDP 24\TYNDP Translator\LEGO_data_templates_Pyomo'

# Path to pypsa data for electric line parameters
pypsa_dir = 'PyPSA_Data'


# Define destination folder where the results should be stored
if simulation_year == 2030:
    if scenario == 'National Trends':
        destination_dir = 'L:\TYNDP 24\Results\Pyomo\V2G_2030_NT'
    elif scenario == 'Distributed Energy':
        destination_dir = 'L:\TYNDP 24\Results\Pyomo\V2G_2030_DE'
    elif scenario == 'Global Ambition':
        destination_dir = 'L:\TYNDP 24\Results\Pyomo\V2G_2030_GA'
    else:
        raise ValueError("Unsupported scenario. Please choose 'National Trends', 'Distributed Energy', or 'Global Ambition'.")
elif simulation_year == 2040:
    if scenario == 'National Trends':
        destination_dir = 'L:\TYNDP 24\Results\Pyomo\V2G_2040_NT'
    elif scenario == 'Distributed Energy':
        destination_dir = 'L:\TYNDP 24\Results\Pyomo\V2G_2040_DE'
    elif scenario == 'Global Ambition':
        destination_dir = 'L:\TYNDP 24\Results\Pyomo\V2G_2040_GA'
    else:
        raise ValueError("Unsupported scenario. Please choose 'National Trends', 'Distributed Energy', or 'Global Ambition'.")
elif simulation_year == 2050:
    if scenario == 'National Trends':
        destination_dir = 'L:\TYNDP 24\Results\Pyomo\V2G_2050_NT'
    elif scenario == 'Distributed Energy':
        destination_dir = 'L:\TYNDP 24\Results\Pyomo\V2G_2050_DE'
    elif scenario == 'Global Ambition':
        destination_dir = 'L:\TYNDP 24\Results\Pyomo\V2G_2050_GA'
    else:
        raise ValueError("Unsupported scenario. Please choose 'National Trends', 'Distributed Energy', or 'Global Ambition'.")
else:
    raise ValueError("Unsupported simulation year. Please choose 2030, 2040, or 2050.")

# Ensure the destination directory exists
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)


## Other parameters




---
## 2. Pre-Tasks


In [ ]:
# Import necessary libraries

import sys
import os
import pandas as pd
import shutil
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from copy import copy
import subprocess

In [ ]:
# Ensure the destination directory exists
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)
else:
    print("Warning: Destination directory already exists. Files may be overwritten.")

In [ ]:
# Define function to check if a file is currently open
def is_file_open(destination_file):
    """Check if a file is open by another application."""
    if os.name == 'nt':  # Windows
        try:
            with open(destination_file, 'r+b') as file:
                pass  # If successful, file is not open elsewhere
            return False
        except PermissionError:
            return True
    else:  # macOS/Linux
        temp_file = os.path.join(os.path.dirname(destination_file), '~$' + os.path.basename(destination_file))
        if os.path.exists(temp_file):
            return True
        else:
            return False

---
## 3. Copying files

### 3.1 Power_Parameters

In [ ]:
# File paths for source data, template, and destination
source_file_other_sources = os.path.join(tyndp_dir, 'Sources_and_other_data.xlsx')
template_file_parameters = os.path.join(template_dir, 'Power_Parameters.xlsx')
destination_file_parameters = os.path.join(destination_dir, 'Power_Parameters.xlsx')

# Remove the existing destination file if it exists to ensure a clean start
if os.path.exists(destination_file_parameters):
    if is_file_open(destination_file_parameters):
        sys.exit(f"File '{destination_file_parameters}' is currently open by another application.")
    os.remove(destination_file_parameters)

# Copy the template file to the destination as a base for modifications
shutil.copy(template_file_parameters, destination_file_parameters)

# Load relevant data from the source file
df_other_sources = pd.read_excel(source_file_other_sources, sheet_name="Power_Parameters", usecols=[1,2,3], header=0)

# Open the destination workbook and select the active worksheet
destination_wb = load_workbook(destination_file_parameters)
destination_ws = destination_wb.active

# Update specific cells in the destination file with values from the source data
destination_ws.cell(row=5, column=3, value=df_other_sources.iloc[2,1])
destination_ws.cell(row=56, column=3, value=df_other_sources.iloc[3,1])

# Save and close the updated file
destination_wb.save(destination_file_parameters)
destination_wb.close()

print(f"File saved successfully at: {destination_file_parameters}")

### 3.2 Power_BusInfo

In [ ]:
# File paths for source data, template, and destination
source_file_businfo = os.path.join(tyndp_dir, 'Nodes/LIST OF NODES.xlsx')
source_file_businfo_offshore = os.path.join(tyndp_dir, 'Line data/ReferenceGrid_Electricity.xlsx')
source_file_other_sources = os.path.join(tyndp_dir, 'Sources_and_other_data.xlsx')
template_file_businfo = os.path.join(template_dir, 'Power_BusInfo.xlsx')
destination_file_businfo = os.path.join(destination_dir, 'Power_BusInfo.xlsx')

# Remove the existing destination file if it exists to ensure a clean start
if os.path.exists(destination_file_businfo):
    if is_file_open(destination_file_businfo):
        sys.exit(f"File '{destination_file_businfo}' is currently open by another application.")
    os.remove(destination_file_businfo)

# Copy the template file to the destination as a base for modifications
shutil.copy(template_file_businfo, destination_file_businfo)

# Load the destination workbook and select the first sheet
destination_wb = load_workbook(destination_file_businfo)
destination_ws = destination_wb.active

# Load and process (onshore) bus data
df_source = pd.read_excel(source_file_businfo, usecols=[0], header=0)  # Read column A (bus names)
df_other_sources = pd.read_excel(source_file_other_sources, sheet_name="Power_BusInfo", usecols=[1,2,3], header=0)

# Write onshore buses to the destination file, starting at row 8 in column C
start_row = 8
for i, value in enumerate(df_source.iloc[:, 0], start=start_row):
    destination_ws[f'C{i}'] = value # Bus name
    destination_ws[f'D{i}'] = f'Zon_{value}' # Zone name

    # Assign static parameter values from df_other_sources
    destination_ws[f'E{i}'] = df_other_sources.iloc[4, 1] # BasVolt
    destination_ws[f'F{i}'] = df_other_sources.iloc[5, 1] # maxVolt
    destination_ws[f'G{i}'] = df_other_sources.iloc[6, 1] # minVolt
    destination_ws[f'H{i}'] = df_other_sources.iloc[7, 1] # Bs
    destination_ws[f'I{i}'] = df_other_sources.iloc[8, 1] # Gs
    destination_ws[f'J{i}'] = df_other_sources.iloc[9, 1] # PowerFactor
    if df_other_sources.iloc[10, 1] != "":
        destination_ws[f'K{i}'] = df_other_sources.iloc[10, 1] # YearCom
    if df_other_sources.iloc[11, 1] != "":
        destination_ws[f'L{i}'] = df_other_sources.iloc[11, 1] # YearDecom
    if df_other_sources.iloc[12, 1] != "":
        destination_ws[f'M{i}'] = df_other_sources.iloc[12, 1] # lat
    if df_other_sources.iloc[13, 1] != "":
        destination_ws[f'N{i}'] = df_other_sources.iloc[12, 1] # lon
    destination_ws[f'O{i}'] = 1 # Whether node is relevant for end result (1) or not (0)
    destination_ws[f'P{i}'] = f"V2G_{simulation_year}_{scenario}" # Which package this belongs to
    destination_ws[f'Q{i}'] = Datasource # Where the data for the entry comes from



# Load and process offshore bus data
df_source_offshore = pd.read_excel(source_file_businfo_offshore, sheet_name="Offshore (for info)", usecols=[0], header=0)

# Extract offshore bus names and split ranges (e.g., "AL00-BA00" → "AL00", "BA00")
bus_list = []
for entry in df_source_offshore.iloc[:, 0].dropna().astype(str):
    bus_list.extend(entry.split('-'))

# Remove duplicates while preserving order
bus_list = list(dict.fromkeys(bus_list))

# Get the list of existing onshore buses
existing_buses = set(df_source.iloc[:, 0].dropna())

# Identify offshore buses that are not already in column B
new_buses = [bus for bus in bus_list if bus not in existing_buses]   

# Append new offshore buses to the destination file
if new_buses:
    for i, value in enumerate(new_buses, start=len(existing_buses) + start_row):
        destination_ws[f'C{i}'] = value # Bus name
        destination_ws[f'D{i}'] = f'Zon_{value}' # Zone name

        # Assign static parameter values from df_other_sources
        destination_ws[f'E{i}'] = df_other_sources.iloc[4, 1] # BasVolt
        destination_ws[f'F{i}'] = df_other_sources.iloc[5, 1] # maxVolt
        destination_ws[f'G{i}'] = df_other_sources.iloc[6, 1] # minVolt
        destination_ws[f'H{i}'] = df_other_sources.iloc[7, 1] # Bs
        destination_ws[f'I{i}'] = df_other_sources.iloc[8, 1] # Gs
        destination_ws[f'J{i}'] = df_other_sources.iloc[9, 1] # PowerFactor
        if df_other_sources.iloc[10, 1] != "":
            destination_ws[f'K{i}'] = df_other_sources.iloc[10, 1] # YearCom
        if df_other_sources.iloc[11, 1] != "":
            destination_ws[f'L{i}'] = df_other_sources.iloc[11, 1] # YearDecom
        if df_other_sources.iloc[12, 1] != "":
            destination_ws[f'M{i}'] = df_other_sources.iloc[12, 1] # lat
        if df_other_sources.iloc[13, 1] != "":
            destination_ws[f'N{i}'] = df_other_sources.iloc[12, 1] # lon
        destination_ws[f'O{i}'] = "1" # Whether node is relevant for end result (1) or not (0)
        destination_ws[f'P{i}'] = f"V2G_{simulation_year}_{scenario}" # Which package this belongs to
        destination_ws[f'Q{i}'] = Datasource # Where the data for the entry comes from

# Save and close the updated destination file
destination_wb.save(destination_file_businfo)
destination_wb.close()

print(f"File saved successfully at: {destination_file_businfo}")


### 3.3 Power_Demand

In [ ]:
# Define file paths for source data, template, and destination
if simulation_year == 2030:
    if scenario == 'National Trends':
        source_file_demand_profiles = os.path.join(tyndp_dir, 'Demand Profiles/NT/Electricity demand profiles/2030_National Trends.xlsx')
    elif scenario == 'Distributed Energy':
        source_file_demand_profiles = os.path.join(tyndp_dir, 'Demand Profiles/DE/2030/ELECTRICITY_MARKET DE 2030.xlsx')
    elif scenario == 'Global Ambition':
        source_file_demand_profiles = os.path.join(tyndp_dir, 'Demand Profiles/GA/2030/ELECTRICITY_MARKET GA 2030.xlsx')
    else:
        raise ValueError("Unsupported scenario. Please choose 'National Trends', 'Distributed Energy', or 'Global Ambition'.")
elif simulation_year == 2040:
    if scenario == 'National Trends':
        source_file_demand_profiles = os.path.join(tyndp_dir, 'Demand Profiles/NT/Electricity demand profiles/2040_National Trends.xlsx')
    elif scenario == 'Distributed Energy':
        source_file_demand_profiles = os.path.join(tyndp_dir, 'Demand Profiles/DE/2040/ELECTRICITY_MARKET DE 2040.xlsx')
    elif scenario == 'Global Ambition':
        source_file_demand_profiles = os.path.join(tyndp_dir, 'Demand Profiles/GA/2030/ELECTRICITY_MARKET GA 2040.xlsx')
    else:
        raise ValueError("Unsupported scenario. Please choose 'National Trends', 'Distributed Energy', or 'Global Ambition'.")
elif simulation_year == 2050:
    if scenario == 'National Trends':
        raise ValueError("National Trends scenario is not available for 2050.")
    elif scenario == 'Distributed Energy':
        source_file_demand_profiles = os.path.join(tyndp_dir, 'Demand Profiles/DE/2050/ELECTRICITY_MARKET DE 2050.xlsx')
    elif scenario == 'Global Ambition':
        source_file_demand_profiles = os.path.join(tyndp_dir, 'Demand Profiles/GA/2050/ELECTRICITY_MARKET GA 2050.xlsx')
    else:
        raise ValueError("Unsupported scenario. Please choose 'National Trends', 'Distributed Energy', or 'Global Ambition'.")
else:
    raise ValueError("Unsupported simulation year. Please choose 2030, 2040, or 2050.")
source_file_other_sources = os.path.join(tyndp_dir, 'Sources_and_other_data.xlsx')
template_file_demand = os.path.join(template_dir, 'Power_Demand.xlsx')
destination_file_demand = os.path.join(destination_dir, 'Power_Demand.xlsx')

# Remove the existing destination file if it exists to ensure a clean start
if os.path.exists(destination_file_demand):
    if is_file_open(destination_file_demand):
        sys.exit(f"File '{destination_file_demand}' is currently open by another application.")
    os.remove(destination_file_demand)

# Copy the template file to the destination as a base for modifications
shutil.copy(template_file_demand, destination_file_demand)

# Validate that the selected year is within the supported range
if cy_year < 1982 or cy_year > 2019:
    raise ValueError("Year must be between 1982 and 2019.")

# Load the demand profiles Excel file (containing multiple sheets for different nodes)
demand_profiles_wb = pd.ExcelFile(source_file_demand_profiles)

# Load additional parameters from another source file
df_other_sources = pd.read_excel(source_file_other_sources, sheet_name="Power_Demand", usecols=[1,2,3], header=0)

# Load the destination workbook where the data will be written
destination_wb = load_workbook(destination_file_demand)
destination_ws = destination_wb.active

# Define the starting row in the destination file for writing demand profiles
row_start = 8

# Process each worksheet (node) in the demand profiles file
for sheet_name in demand_profiles_wb.sheet_names:
    print(f"Start copying demand for {sheet_name}.")
    
    # Load the time-series data for the node, skipping metadata rows
    node_df = demand_profiles_wb.parse(sheet_name, skiprows=11, nrows=8760)

    # Identify year columns (assuming data for 1982-2019 starts from column E)
    year_columns = node_df.columns[4:len(node_df.columns)]
    
    # Check if the selected year is available in the dataset
    if cy_year not in year_columns:
        print(f"Year {cy_year} not found in sheet {sheet_name}. Skipping node.")
        continue
    
    # Get the column index corresponding to the selected year
    year_col_idx = year_columns.get_loc(cy_year) + 4  # Convert relative index to absolute column index

    # Extract the hourly demand time series for the selected year
    time_series = node_df.iloc[:, year_col_idx]

    # Write periode and node name value to the destination file

    destination_ws[f'C{row_start}'] = df_other_sources.iloc[2, 1] # Periode
    destination_ws[f'D{row_start}'] = sheet_name # Node name
    destination_ws[f'E{i}'] = f"V2G_{simulation_year}_{scenario}" # Which package this belongs to
    destination_ws[f'F{i}'] = Datasource # Where the data for the entry comes from
    # Write the demand values (8760 hours) into the corresponding row, starting from column E
    for i, demand in enumerate(time_series, start=7):
        destination_ws.cell(row=row_start, column=i, value=round(demand,2))
    
    # Move to the next row for the next node
    row_start += 1

# Save and close the modified workbook to the destination file
destination_wb.save(destination_file_demand)
destination_wb.close()

print(f"Data saved successfully at: {destination_file_demand}")

### 3.4 Power_Inflows

In [ ]:
# Define file paths for source data, template, and destination
if simulation_year == 2030:
    folder_source_files_inflows = os.path.join(tyndp_dir, 'Hydro Inflows/2030')
elif simulation_year == 2040:
    folder_source_files_inflows = os.path.join(tyndp_dir, 'Hydro Inflows/2040')
elif simulation_year == 2050:
    folder_source_files_inflows = os.path.join(tyndp_dir, 'Hydro Inflows/2050')
else:
    raise ValueError("Unsupported simulation year. Please choose 2030, 2040, or 2050.")

source_file_other_sources = os.path.join(tyndp_dir, 'Sources_and_other_data.xlsx')
template_file_inflows = os.path.join(template_dir, 'Power_Inflows.xlsx')
destination_file_inflows = os.path.join(destination_dir, 'Power_Inflows.xlsx')

# Remove the existing destination file if it exists to ensure a clean start
if os.path.exists(destination_file_inflows):
    if is_file_open(destination_file_inflows):
        sys.exit(f"File '{destination_file_inflows}' is currently open by another application.")
    os.remove(destination_file_inflows)

# Copy the template file to the destination as a base for modifications
shutil.copy(template_file_inflows, destination_file_inflows)

# Load additional parameters from another source file
df_other_sources = pd.read_excel(source_file_other_sources, sheet_name="Power_Inflows", usecols=[1,2,3], header=0)

# Load the destination workbook where the processed data will be stored
destination_wb = load_workbook(destination_file_inflows)
destination_ws = destination_wb.active

# Retrieve all relevant inflow data files from the specified folder
inflows_files = sorted(f for f in os.listdir(folder_source_files_inflows) if (f.startswith("PEMMDB") and f.endswith(".xlsx")))

# Define the starting row in the destination file for writing network data
row_start = 7

# Process each inflow file
for filename in inflows_files:
    file_path = os.path.join(folder_source_files_inflows, filename)  # Construct full file path
    if os.path.isfile(file_path):  # Ensure it's a file

        # Load the inflows Excel file
        inflows_wb = pd.ExcelFile(file_path)
        print(file_path)

        # Load different types of inflow data from the workbook
        ror_df = inflows_wb.parse('Run of River - Year Dependent', skiprows=1, nrows=365)
        pondage_df = inflows_wb.parse('Pondage - Year Dependent', skiprows=1, nrows=365)
        hs_df = inflows_wb.parse('Reservoir - Year Dependent', skiprows=1, nrows=53)
        hps_df = inflows_wb.parse('PS Open - Year Dependent', skiprows=1, nrows=53)
        hcps_df = inflows_wb.parse('PS Closed - Year Dependent', skiprows=1, nrows=53)

        # Identify year columns (assuming data for 1982-2019 starts from column D)
        year_columns = ror_df.columns[3:]
        
        # Check if the selected year is available in the dataset
        if cy_year not in year_columns:
            print(f"Year {cy_year} not found in sheet {filename}. Skipping node.")
            continue

        # Get the column index corresponding to the selected year
        year_col_idx = year_columns.get_loc(cy_year) + 3  # Adjust index to match actual column position

        # Extract the inflow time series for the selected year, replacing NaNs with zeros
        time_series_ror = ror_df.iloc[:, year_col_idx].fillna(0)
        time_series_pondage = pondage_df.iloc[:, year_col_idx].fillna(0)
        time_series_hs = hs_df.iloc[:, year_col_idx].fillna(0)
        time_series_hps = hps_df.iloc[:, year_col_idx].fillna(0)
        time_series_hcps = hcps_df.iloc[:, year_col_idx].fillna(0)

        # Combine Run-of-River (ROR) and Pondage inflow data
        time_series_ror_pondage = time_series_ror + time_series_pondage

        # Define inflow categories and their respective time series
        inflow_categories = {
            'ROR': time_series_ror_pondage,
            'HS': time_series_hs,
            'HPS': time_series_hps,
            'HCPS': time_series_hcps
        }

        # Process and write data for each inflow category
        for inflow_type, series in inflow_categories.items():
            if series.sum() == 0:  # Skip writing if the entire time series is zero
                continue

            # Write periode and power plant name value to the destination file
            destination_ws[f'B{row_start}'] = df_other_sources.iloc[2, 1] # Periode
            destination_ws[f'C{row_start}'] = filename.split('_')[1] + inflow_type # Power plant name
            print(f"Writing {inflow_type} data...")

            # Define the starting column in the destination file for writing inflows
            column_start = 4

            # RoR inflows are provided as a daily profile
            if inflow_type == 'ROR':
                # Distribute daily inflows evenly across 24 hours
                for inflow in series:  
                    for h in range(24):  # Repeat for each hour of the day
                        destination_ws.cell(row=row_start, column=column_start, value=round(inflow*1000 / 24,4))
                        column_start += 1
            
            # HS, HPS and HS inflows are provided as weekly profiles
            elif inflow_type in ['HS', 'HPS', 'HCPS']:
                hour_count = 0  # Counter to track written hours
                previous_inflow = 0 # Store previous week's inflow value
                
                for inflow in series:
                    for h in range(7 * 24):  # Distribute weekly inflows across 7 days (168 hours)
                        if hour_count >= 8760:  # Stop once reaching 8760 hours
                            break

                        # If in the last week and inflow is missing, use previous week's value
                        if hour_count >= 8736 and inflow == 0:
                            inflow = previous_inflow if previous_inflow is not None else 0
                        
                        # Write the inflow value to the corresponding cell
                        destination_ws.cell(row=row_start, column=column_start, value=round(inflow*1000 / (7 * 24),4))
                        
                        previous_inflow = inflow # Store current inflow as reference for next iteration
                        column_start += 1
                        hour_count += 1

                    if hour_count >= 8760:  # Stop processing once 8760 hours are written
                        break
        
            # Move to the next row for the next inflow category
            row_start += 1

# Save and close the modified workbook to the destination file
destination_wb.save(destination_file_inflows)
destination_wb.close()

print(f"Data saved successfully at: {destination_file_inflows}")


### 3.5 Power_Network

In [ ]:
# Define file paths for source data, template, and destination file
source_file_network = os.path.join(tyndp_dir, 'Line data/ReferenceGrid_Electricity.xlsx')
source_file_other_sources = os.path.join(tyndp_dir, 'Sources_and_other_data.xlsx')
template_file_network = os.path.join(template_dir, 'Power_Network.xlsx')
destination_file_network = os.path.join(destination_dir, 'Power_Network.xlsx')

# If the destination file exists, remove it to ensure a clean start
if os.path.exists(destination_file_network):
    if is_file_open(destination_file_network):
        sys.exit(f"File '{destination_file_network}' is currently open by another application.")
    os.remove(destination_file_network)

# Copy the template file to the destination for modification
shutil.copy(template_file_network, destination_file_network)

# Load the destination workbook and select the active worksheet
destination_wb = load_workbook(destination_file_network)
destination_ws = destination_wb.active

# Load onshore transmission network data
df_source = pd.read_excel(source_file_network, usecols=[0,1,2], header=0, sheet_name="2030")  # Read column A

# Load additional parameters from another source file
df_other_sources = pd.read_excel(source_file_other_sources, sheet_name="Power_Network", usecols=[1,2,3], header=0)

# Replace column 1 values with the maximum of column 1 and column 2 to get the max transport capacity
df_source.iloc[:, 1] = df_source.iloc[:, 1].combine(df_source.iloc[:, 2], max)

# Remove column C as it's no longer needed
df_source.drop(df_source.columns[2], axis=1, inplace=True)

# Keep only rows where the transmission capacity (column B) is non-zero
df_source = df_source[df_source.iloc[:, 1] != 0]

# Define the starting row in the destination file for writing NTC data
start_row = 8

# Write processed onshore network data to the destination file
for i, value in enumerate(df_source.iloc[:, 0], start=start_row):
    destination_ws[f'C{i}'] = value.split('-')[0]  # Bus from name in column B
    destination_ws[f'D{i}'] = value.split('-')[1]  # Bus to name in column C

    # Assign predefined static values from the other sources file
    destination_ws[f'E{i}'] = df_other_sources.iloc[4, 1] # Circuit
    destination_ws[f'I{i}'] = df_other_sources.iloc[9, 1] # TapAngle
    destination_ws[f'J{i}'] = df_other_sources.iloc[10, 1] # TapRatio
    destination_ws[f'K{i}'] = df_source.iloc[i-start_row, 1] # Pmax
    destination_ws[f'L{i}'] = 1 - df_other_sources.iloc[5, 1] # InService (inverted) since it is changed in the Pyomo model

# Save the modified workbook
destination_wb.save(destination_file_network)

# Load offshore transmission network data
df_source_offshore = pd.read_excel(source_file_network, usecols=[0,1,2], header=0, sheet_name="Offshore (for info)")  # Read column A

# Replace column 1 values with the maximum of column 1 and column 2 to get the max transport capacity
df_source_offshore.iloc[:, 1] = df_source_offshore.iloc[:, 1].combine(df_source_offshore.iloc[:, 2], max)

# Remove column C as it's no longer needed
df_source_offshore.drop(df_source_offshore.columns[2], axis=1, inplace=True)

# Keep only rows where the transmission capacity (column B) is non-zero
df_source_offshore = df_source_offshore[df_source_offshore.iloc[:, 1] != 0]

# Write processed offshore network data to the destination file
for i, value in enumerate(df_source_offshore.iloc[:, 0], start=len(df_source) + start_row):
    destination_ws[f'C{i}'] = value.split('-')[0]  # Bus from name in column B
    destination_ws[f'D{i}'] = value.split('-')[1]  # Bus to name in column C

    # Assign predefined static values from the other sources file
    destination_ws[f'E{i}'] = df_other_sources.iloc[4, 1] # Circuit
    destination_ws[f'I{i}'] = df_other_sources.iloc[9, 1] # TapAngle
    destination_ws[f'J{i}'] = df_other_sources.iloc[10, 1] # TapRatio
    destination_ws[f'K{i}'] = df_source_offshore.iloc[i-start_row-len(df_source), 1] # Pmax
    destination_ws[f'L{i}'] = 1 - df_other_sources.iloc[5, 1] # InService (inverted) since it is changed in the Pyomo model
# Save and close the final modified workbook
destination_wb.save(destination_file_network)
destination_wb.close()

print(f"Data saved successfully at: {destination_file_network}")

### 3.6 Power_NTC

In [ ]:
# # Define file paths for source data, template, and destination file
# source_file_ntc = os.path.join(tyndp_dir, 'Line data/ReferenceGrid_Electricity.xlsx')
# template_file_ntc = os.path.join(template_dir, 'Power_NTC.xlsx')
# destination_file_ntc = os.path.join(destination_dir, 'Power_NTC.xlsx')

# # If the destination file exists, remove it to ensure a clean start
# if os.path.exists(destination_file_ntc):
#     if is_file_open(destination_file_ntc):
#         sys.exit(f"File '{destination_file_ntc}' is currently open by another application.")
#     os.remove(destination_file_ntc)

# # Copy the template file to the destination for modification
# shutil.copy(template_file_ntc, destination_file_ntc)

# # Load the destination workbook and select the active worksheet
# destination_wb = load_workbook(destination_file_ntc)
# destination_ws = destination_wb.active

# # Load and process onshore transmission network data
# df_source = pd.read_excel(source_file_ntc, usecols=[0,1,2], header=0, sheet_name="2030")  # Read column A

# # Keep only the rows where both columns B and C are not zero
# df_source = df_source[~((df_source.iloc[:, 1] == 0) & (df_source.iloc[:, 2] == 0))]

# # Define the starting row in the destination file for writing inflow data
# start_row = 7

# # Write the processed onshore network data to the destination file
# for i, value in enumerate(df_source.iloc[:, 0], start=start_row):
#     # Assign 'from' and 'to' zone names, and their respective transmission capacities
#     destination_ws[f'B{i}'] = 'Zon_' + value.split('-')[0]  # FromZone
#     destination_ws[f'C{i}'] = 'Zon_' + value.split('-')[1]  # ToZone
#     destination_ws[f'D{i}'] = df_source.iloc[i-start_row, 1] # Pmax

# # Write the reverse direction (from 'to' to 'from') with respective transmission capacities
# for i, value in enumerate(df_source.iloc[:, 0], start=len(df_source) + start_row):
#     destination_ws[f'B{i}'] = 'Zon_' + value.split('-')[1]  # FromZone
#     destination_ws[f'C{i}'] = 'Zon_' + value.split('-')[0]  # ToZone
#     destination_ws[f'D{i}'] = df_source.iloc[i-start_row-len(df_source) , 2] # Pmax


# # Save the modified workbook to the destination
# destination_wb.save(destination_file_ntc)

# # Load and process offshore transmission network data
# df_source_offshore = pd.read_excel(source_file_ntc, usecols=[0,1,2], header=0, sheet_name="Offshore (for info)")  # Read column A

# # Keep only the rows where both columns B and C are not zero
# df_source_offshore = df_source_offshore[~((df_source_offshore.iloc[:, 1] == 0) & (df_source_offshore.iloc[:, 2] == 0))]

# # Write the processed offshore network data to the destination file
# for i, value in enumerate(df_source_offshore.iloc[:, 0], start=2*len(df_source) + start_row):
#     # Assign 'from' and 'to' zone names, and their respective transmission capacities
#     destination_ws[f'B{i}'] = 'Zon_' + value.split('-')[0]  # FromZone
#     destination_ws[f'C{i}'] = 'Zon_' + value.split('-')[1]  # ToZone
#     destination_ws[f'D{i}'] = df_source_offshore.iloc[i-start_row-2*len(df_source), 1] # Pmax

# # Write the reverse direction (from 'to' to 'from') with respective transmission capacities
# for i, value in enumerate(df_source_offshore.iloc[:, 0], start=2*len(df_source) + len(df_source_offshore) + start_row):
#     destination_ws[f'B{i}'] = 'Zon_' + value.split('-')[1]  # FromZone
#     destination_ws[f'C{i}'] = 'Zon_' + value.split('-')[0]  # ToZone
#     destination_ws[f'D{i}'] = df_source_offshore.iloc[i-start_row-2*len(df_source)-len(df_source_offshore), 2] # Pmax

# # Save and close the final modified workbook to the destination
# destination_wb.save(destination_file_ntc)
# destination_wb.close()

# print(f"Data saved successfully at: {destination_file_ntc}")

### 3.7 Power_RoR, Power_Storage, Power_VRES & Power_ThermalGen

In [ ]:
# Define file paths for source data, template files, and destination files
if simulation_year == 2030:
    folder_source_files_generators = os.path.join(tyndp_dir, 'PEMMDB2/2030/')
elif simulation_year == 2040:
    folder_source_files_generators = os.path.join(tyndp_dir, 'PEMMDB2/2040/')   
elif simulation_year == 2050:
    folder_source_files_generators = os.path.join(tyndp_dir, 'PEMMDB2/2050/')
else:
    raise ValueError("Unsupported simulation year. Please choose 2030, 2040, or 2050.")

source_file_other_sources = os.path.join(tyndp_dir, 'Sources_and_other_data.xlsx')
source_file_prices = os.path.join(tyndp_dir, 'Prices/2023 06 22 TYNDP 2024 Commodity prices Final.xlsx')

# Define template and destination file mappings for different power generation types
files = {
    "RoR": (os.path.join(template_dir, 'Power_RoR.xlsx'), os.path.join(destination_dir, 'Power_RoR.xlsx')),
    "Storage": (os.path.join(template_dir, 'Power_Storage.xlsx'), os.path.join(destination_dir, 'Power_Storage.xlsx')),
    "VRES": (os.path.join(template_dir, 'Power_VRES.xlsx'), os.path.join(destination_dir, 'Power_VRES.xlsx')),
    "Thermal": (os.path.join(template_dir, 'Power_ThermalGen.xlsx'), os.path.join(destination_dir, 'Power_ThermalGen.xlsx')),
}

# Load price data from the source Excel file
if simulation_year == 2030:
    df_prices = pd.read_excel(source_file_prices, sheet_name="Matrix 2024", usecols=[1, 2, 3], header=0)
elif simulation_year == 2040:
    df_prices = pd.read_excel(source_file_prices, sheet_name="Matrix 2024", usecols=[1, 2, 4], header=0)
elif simulation_year == 2050:
    df_prices = pd.read_excel(source_file_prices, sheet_name="Matrix 2024", usecols=[1, 2, 5], header=0)
else:
    raise ValueError("Unsupported simulation year. Please choose 2030, 2040, or 2050.")

# Load data for various power generation sources
df_other_sources_RoR = pd.read_excel(source_file_other_sources, sheet_name="Power_RoR", usecols=[1,2,3], header=0)
df_other_sources_Storage_HS = pd.read_excel(source_file_other_sources, sheet_name="Power_Storage_HS", usecols=[1,2,3], header=0)
df_other_sources_Storage_HPS = pd.read_excel(source_file_other_sources, sheet_name="Power_Storage_HPS", usecols=[1,2,3], header=0)
df_other_sources_Storage_HPSCL = pd.read_excel(source_file_other_sources, sheet_name="Power_Storage_HPSCL", usecols=[1,2,3], header=0)
df_other_sources_Storage_BESS = pd.read_excel(source_file_other_sources, sheet_name="Power_Storage_BESS", usecols=[1,2,3], header=0)
df_other_sources_VRES_Wind = pd.read_excel(source_file_other_sources, sheet_name="Power_VRES_Wind", usecols=[1,2,3], header=0)
df_other_sources_VRES_WindOff = pd.read_excel(source_file_other_sources, sheet_name="Power_VRES_WindOff", usecols=[1,2,3], header=0)
df_other_sources_VRES_PV = pd.read_excel(source_file_other_sources, sheet_name="Power_VRES_PV", usecols=[1,2,3], header=0)
df_other_sources_VRES_CSP = pd.read_excel(source_file_other_sources, sheet_name="Power_VRES_CSP", usecols=[1,2,3], header=0)
df_other_sources_VRES_OtherRES = pd.read_excel(source_file_other_sources, sheet_name="Power_VRES_OtherRES", usecols=[1,2,3], header=0)
df_other_sources_Thermal_Nuclear = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_Nuclear", usecols=[1,2,3], header=0)
df_other_sources_Thermal_CoalOld1 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_CoalOld1", usecols=[1,2,3], header=0)
df_other_sources_Thermal_CoalOld2 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_CoalOld2", usecols=[1,2,3], header=0)
df_other_sources_Thermal_CoalNew = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_CoalNew", usecols=[1,2,3], header=0)
df_other_sources_Thermal_CoalCCS = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_CoalCCS", usecols=[1,2,3], header=0)
df_other_sources_Thermal_LigniteOld1 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_LigniteOld1", usecols=[1,2,3], header=0)
df_other_sources_Thermal_LigniteOld2 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_LigniteOld2", usecols=[1,2,3], header=0)
df_other_sources_Thermal_LigniteNew = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_LigniteNew", usecols=[1,2,3], header=0)
df_other_sources_Thermal_LigniteCCS = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_LigniteCCS", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasConvOld1 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasConvOld1", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasConvOld2 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasConvOld2", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasCCGTOld1 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasCCGTOld1", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasCCGTOld2 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasCCGTOld2", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasCCGTNew = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasCCGTNew", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasCCGTCCS = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasCCGTCCS", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasOCGTOld = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasOCGTOld", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasOCGTNew = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasOCGTNew", usecols=[1,2,3], header=0)
df_other_sources_Thermal_LightOil = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_LightOil", usecols=[1,2,3], header=0)
df_other_sources_Thermal_HeavyOilOld1 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_HeavyOilOld1", usecols=[1,2,3], header=0)
df_other_sources_Thermal_HeavyOilOld2 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_HeavyOilOld2", usecols=[1,2,3], header=0)
df_other_sources_Thermal_OilShaleOld = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_OilShaleOld", usecols=[1,2,3], header=0)
df_other_sources_Thermal_OilShaleNew = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_OilShaleNew", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasCCGTPresent1 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasCCGTPresent1", usecols=[1,2,3], header=0)
df_other_sources_Thermal_GasCCGTPresent2 = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_GasCCGTPresent2", usecols=[1,2,3], header=0)
df_other_sources_Thermal_OtherNonRES = pd.read_excel(source_file_other_sources, sheet_name="Power_Thermal_OtherNonRES", usecols=[1,2,3], header=0)

# Dictionary to store loaded workbooks and worksheets
workbooks = {}
worksheets = {}

# Iterate through each power generation type and prepare destination files
for key, (template_file, destination_file) in files.items():
    # If the destination file already exists, remove it to start fresh
    if os.path.exists(destination_file):
        if is_file_open(destination_file):
            sys.exit(f"File '{destination_file}' is currently open by another application.")
        os.remove(destination_file)

    # Copy the template file to the destination
    shutil.copy(template_file, destination_file)

    # Load the copied file as a workbook and select the active worksheet
    destination_wb = load_workbook(destination_file)
    destination_ws = destination_wb.active

    # Store the workbook and worksheet in the dictionaries for later use
    workbooks[key] = destination_wb
    worksheets[key] = destination_ws

# Get all the generator files that start with "PEMMDB" and end with ".xlsx"
generators_files = sorted(f for f in os.listdir(folder_source_files_generators) if (f.startswith("PEMMDB") and f.endswith(".xlsx")))

# Define the starting row in the destination file for writing generator data

# Initialize row counters for each generator type
cur_row_ror = 0
cur_row_storage = 0
cur_row_vres = 0
cur_row_thermal = 0

# Iterate through each generator file
for filename in generators_files:
    file_path = os.path.join(folder_source_files_generators, filename)  # Full file path
    if os.path.isfile(file_path):  # Ensure it's a valid file

        # Load the generator data from the Excel file
        generators_wb = pd.ExcelFile(file_path)
        print(file_path)

        # Load individual sheets from the generator Excel file
        hydro_df = generators_wb.parse('Hydro')
        thermal_df = generators_wb.parse('Thermal')
        otherNonRes_df = generators_wb.parse('Other Non-RES')
        battery_df = generators_wb.parse('Battery')
        wind_df = generators_wb.parse('Wind')
        solar_df = generators_wb.parse('Solar')
        otherRes_df = generators_wb.parse('Other RES')

        # Process data for Run-of-River (RoR) plants
        if round((hydro_df.iloc[7, 1]+hydro_df.iloc[10, 1]),1) > 0:
            row_start = 7
            # Add data for RoR generation to the worksheet
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=2, value = filename.split('_')[1]+'ROR') # PP Name
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=3, value = df_other_sources_RoR.iloc[3, 1]) # Tec
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=4, value = filename.split('_')[1]) # Node/Businfo
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=5, value = df_other_sources_RoR.iloc[5, 1]) # ExisUnit
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=6, value = round(hydro_df.iloc[7, 1]+hydro_df.iloc[10, 1],1)) # MaxProd
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=7, value = df_other_sources_RoR.iloc[7, 1]) # MinProd
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=8, value = df_other_sources_RoR.iloc[8, 1]) # MaxCons
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=9, value = df_other_sources_RoR.iloc[9, 1]) # DisEffic
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=10, value = df_other_sources_RoR.iloc[10, 1]) # ChEffic
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=11, value = df_other_sources_RoR.iloc[11, 1]) # Qmax
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=12, value = df_other_sources_RoR.iloc[12, 1]) # Qmin
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=14, value = df_other_sources_RoR.iloc[14, 1]) # MinReserve
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=15, value = df_other_sources_RoR.iloc[15, 1]) # IniReserve
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=16, value = df_other_sources_RoR.iloc[16, 1]) # IsHydro
            worksheets['RoR'].cell(row=row_start+cur_row_ror, column=22, value = df_other_sources_RoR.iloc[22, 1]) # Ene2PowRatio
            cell_17 = worksheets['RoR'].cell(row=row_start + cur_row_ror, column=17) #OMVarCost
            if cell_17.value is None:
                cell_17.value = 0
            cell_18 = worksheets['RoR'].cell(row=row_start + cur_row_ror, column=18) #EnableInvest
            if cell_18.value is None:
                cell_18.value = 0
            cell_19 = worksheets['RoR'].cell(row=row_start + cur_row_ror, column=19) #MaxInvest   
            if cell_19.value is None:
                cell_19.value = 0
            cell_20 = worksheets['RoR'].cell(row=row_start + cur_row_ror, column=20) #InvestCostPerMW
            if cell_20.value is None:
                cell_20.value = 0
            cell_21 = worksheets['RoR'].cell(row=row_start + cur_row_ror, column=21) #InvestCostPerMWh
            if cell_21.value is None:
                cell_21.value = 0

            cur_row_ror += 1 # Move to the next row
        row_start = 8
        # Process data for Hydro Storage (HS)
        if round(hydro_df.iloc[13, 1],1) > 0:
            # Add data for Hydro Storage to the worksheet
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=2, value = filename.split('_')[1]+'HS') # PP Name
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=3, value = df_other_sources_Storage_HS.iloc[3, 1]) # Tec
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=4, value = filename.split('_')[1]) # Node/Businfo
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=5, value = df_other_sources_Storage_HS.iloc[5, 1]) # ExisUnit
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=6, value = round(hydro_df.iloc[13, 1],1)) # MaxProd
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=7, value = df_other_sources_Storage_HS.iloc[7, 1]) # MinProd
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=8, value = df_other_sources_Storage_HS.iloc[8, 1]) # MaxCons
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=9, value = df_other_sources_Storage_HS.iloc[9, 1]) # DisEffic
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=10, value = df_other_sources_Storage_HS.iloc[10, 1]) # ChEffic
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=11, value = df_other_sources_Storage_HS.iloc[11, 1]) # Qmax
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=12, value = df_other_sources_Storage_HS.iloc[12, 1]) # Qmin
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=14, value = df_other_sources_Storage_HS.iloc[14, 1]) # MinReserve
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=15, value = df_other_sources_Storage_HS.iloc[15, 1]) # IniReserve
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=16, value = df_other_sources_Storage_HS.iloc[16, 1]) # IsHydro
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=17, value = df_other_sources_Storage_HS.iloc[17, 1]) # OMVarCost
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=22, value = round((hydro_df.iloc[12, 1]*1000)/hydro_df.iloc[13, 1],1)) # Ene2PowRatio
            cell_18 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=18) # EnableInvest
            if cell_18.value is None:
                cell_18.value = 0
            cell_19 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=19) # MaxInvest
            if cell_19.value is None:
                cell_19.value = 0
            cell_20 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=20) # InvestCostPerMW
            if cell_20.value is None:
                cell_20.value = 0
            cell_21 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=21) # InvestCostPerMWh
            if cell_21.value is None:
                cell_21.value = 0

            cur_row_storage += 1 # Move to the next row
        
        # Process data for Hydro Pumped Storage (HPS)
        if round(hydro_df.iloc[16, 1],1) > 0:
            # Add data for Hydro Pumped Storage to the worksheet
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=2, value = filename.split('_')[1]+'HPS') # PP Name
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=3, value = df_other_sources_Storage_HPS.iloc[3, 1]) # Tec
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=4, value = filename.split('_')[1]) # Node/Businfo
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=5, value = df_other_sources_Storage_HPS.iloc[5, 1]) # ExisUnit
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=6, value = round(hydro_df.iloc[16, 1],1)) # MaxProd
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=7, value = df_other_sources_Storage_HPS.iloc[7, 1]) # MinProd
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=8, value = -round(hydro_df.iloc[17, 1],1)) # MaxCons
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=9, value = df_other_sources_Storage_HPS.iloc[9, 1]) # DisEffic
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=10, value = df_other_sources_Storage_HPS.iloc[10, 1]) # ChEffic
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=11, value = df_other_sources_Storage_HPS.iloc[11, 1]) # Qmax
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=12, value = df_other_sources_Storage_HPS.iloc[12, 1]) # Qmin
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=14, value = df_other_sources_Storage_HPS.iloc[14, 1]) # MinReserve
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=15, value = df_other_sources_Storage_HPS.iloc[15, 1]) # IniReserve
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=16, value = df_other_sources_Storage_HPS.iloc[16, 1]) # IsHydro
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=17, value = df_other_sources_Storage_HPS.iloc[17, 1]) # OMVarCost
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=22, value = round((hydro_df.iloc[15, 1]*1000)/hydro_df.iloc[16, 1],1)) # Ene2PowRatio
            cell_18 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=18) # EnableInvest
            if cell_18.value is None:
                cell_18.value = 0
            cell_19 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=19) # MaxInvest
            if cell_19.value is None:
                cell_19.value = 0
            cell_20 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=20) # InvestCostPerMW
            if cell_20.value is None:
                cell_20.value = 0
            cell_21 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=21) # InvestCostPerMWh
            if cell_21.value is None:
                cell_21.value = 0

            cur_row_storage += 1 # Move to the next row

        # Process data for Hydro Pumped Storage Closed Loop (HPSCl)
        if round(hydro_df.iloc[20, 1],1) > 0:
            # Add data for Hydro Pumped Storage Closed Loop to the worksheet
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=2, value = filename.split('_')[1]+'HPSCL') # PP Name
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=3, value = df_other_sources_Storage_HPSCL.iloc[3, 1]) # Tec
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=4, value = filename.split('_')[1]) # Node/Businfo
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=5, value = df_other_sources_Storage_HPSCL.iloc[5, 1]) # ExisUnit
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=6, value = round(hydro_df.iloc[20, 1],1)) # MaxProd
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=7, value = df_other_sources_Storage_HPSCL.iloc[7, 1]) # MinProd
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=8, value = -round(hydro_df.iloc[21, 1],1)) # MaxCons
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=9, value = df_other_sources_Storage_HPSCL.iloc[9, 1]) # DisEffic
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=10, value = df_other_sources_Storage_HPSCL.iloc[10, 1]) # ChEffic
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=11, value = df_other_sources_Storage_HPSCL.iloc[11, 1]) # Qmax
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=12, value = df_other_sources_Storage_HPSCL.iloc[12, 1]) # Qmin
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=14, value = df_other_sources_Storage_HPSCL.iloc[14, 1]) # MinReserve
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=15, value = df_other_sources_Storage_HPSCL.iloc[15, 1]) # IniReserve
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=16, value = df_other_sources_Storage_HPSCL.iloc[16, 1]) # IsHydro
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=17, value = df_other_sources_Storage_HPSCL.iloc[17, 1]) # OMVarCost
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=22, value = round((hydro_df.iloc[19, 1]*1000)/hydro_df.iloc[20, 1],1)) # Ene2PowRatio
            cell_18 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=18) # EnableInvest
            if cell_18.value is None:
                cell_18.value = 0
            cell_19 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=19) # MaxInvest
            if cell_19.value is None:
                cell_19.value = 0
            cell_20 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=20) # InvestCostPerMW
            if cell_20.value is None:
                cell_20.value = 0
            cell_21 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=21) # InvestCostPerMWh
            if cell_21.value is None:
                cell_21.value = 0

            cur_row_storage += 1 # Move to the next row

        # Process data for Batteries (BESS)
        if round(battery_df.iloc[10, 2],1) > 0:
            # Add data for Batteries to the worksheet
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=2, value = filename.split('_')[1]+'BESS') # PP Name
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=3, value = df_other_sources_Storage_BESS.iloc[3, 1]) # Tec
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=4, value = filename.split('_')[1]) # Node/Businfo
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=5, value = df_other_sources_Storage_BESS.iloc[5, 1]) # ExisUnit
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=6, value = round(battery_df.iloc[10, 2],1)) # MaxProd
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=7, value = df_other_sources_Storage_BESS.iloc[7, 1]) # MinProd
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=8, value = round(battery_df.iloc[10, 3],1)) # MaxCons
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=9, value = battery_df.iloc[10, 6]) # DisEffic
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=10, value = battery_df.iloc[10, 6]) # ChEffic
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=11, value = df_other_sources_Storage_BESS.iloc[11, 1]) # Qmax
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=12, value = df_other_sources_Storage_BESS.iloc[12, 1]) # Qmin
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=14, value = df_other_sources_Storage_BESS.iloc[14, 1]) # MinReserve
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=15, value = df_other_sources_Storage_BESS.iloc[15, 1]) # IniReserve
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=17, value = df_other_sources_Storage_BESS.iloc[16, 1]) # IsHydro
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=17, value = df_other_sources_Storage_BESS.iloc[17, 1]) # OMVarCost
            worksheets['Storage'].cell(row=row_start+cur_row_storage, column=22, value = round((battery_df.iloc[10, 4])/battery_df.iloc[10, 2],1)) # Ene2PowRatio
            cell_18 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=18) # EnableInvest
            if cell_18.value is None:
                cell_18.value = 0
            cell_19 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=19) # MaxInvest
            if cell_19.value is None:
                cell_19.value = 0
            cell_20 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=20) # InvestCostPerMW
            if cell_20.value is None:
                cell_20.value = 0
            cell_21 = worksheets['Storage'].cell(row=row_start + cur_row_storage, column=21) # InvestCostPerMWh
            if cell_21.value is None:
                cell_21.value = 0

            cur_row_storage += 1 # Move to the next row

        # Process data for Wind power plants
        if round(wind_df.iloc[6, 1],1) > 0:
            # Add data for Wind power plants to the worksheet
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=3, value = filename.split('_')[1]+'WIND') # PP Name
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=4, value = df_other_sources_VRES_Wind.iloc[3, 1]) # Tec
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=5, value = filename.split('_')[1]) # Node/Businfo
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=6, value = df_other_sources_VRES_Wind.iloc[5, 1]) # ExisUnit
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=7, value = round(wind_df.iloc[6, 1]*1000,1)) # MaxProd
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=8, value = df_other_sources_VRES_Wind.iloc[7, 1]) # EnableInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=9, value = df_other_sources_VRES_Wind.iloc[8, 1]) # MaxInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=11, value = df_other_sources_VRES_Wind.iloc[10, 1]) # OMVarCost
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=20, value=f"V2G_{simulation_year}_{scenario}")  # Which package this belongs to
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=21, value=Datasource)  # Where the data for the entry comes from
            cell_10 = worksheets['VRES'].cell(row=row_start + cur_row_vres, column=10) # InvestCost
            if cell_10.value is None:
                cell_10.value = 0
            cur_row_vres += 1 # Move to the next row

        # Process data for Wind Offshore power plants
        if round(wind_df.iloc[7, 1],1) > 0:
            # Add data for Wind Offshore power plants to the worksheet
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=3, value = filename.split('_')[1]+'WINDOFF') # PP Name
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=4, value = df_other_sources_VRES_WindOff.iloc[3, 1]) # Tec
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=5, value = filename.split('_')[1]) # Node/Businfo
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=6, value = df_other_sources_VRES_Wind.iloc[5, 1]) # ExisUnit
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=7, value = round(wind_df.iloc[6, 1]*1000,1)) # MaxProd
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=8, value = df_other_sources_VRES_WindOff.iloc[7, 1]) # EnableInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=9, value = df_other_sources_VRES_WindOff.iloc[8, 1]) # MaxInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=11, value = df_other_sources_VRES_WindOff.iloc[10, 1]) # OMVarCost
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=20, value=f"V2G_{simulation_year}_{scenario}")  # Which package this belongs to
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=21, value=Datasource)  # Where the data for the entry comes from
            cell_10 = worksheets['VRES'].cell(row=row_start + cur_row_vres, column=10) # InvestCost
            if cell_10.value is None:
                cell_10.value = 0

            cur_row_vres += 1 # Move to the next row

        # Process data for Photovoltaic (PV) power plants
        if round(solar_df.iloc[7, 1]+solar_df.iloc[8, 1],1) > 0:
            # Add data for Photovoltaik power plants to the worksheet
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=3, value = filename.split('_')[1]+'PV') # PP Name
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=4, value = df_other_sources_VRES_PV.iloc[3, 1]) # Tec
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=5, value = filename.split('_')[1]) # Node/Businfo
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=6, value = df_other_sources_VRES_PV.iloc[5, 1]) # ExisUnit
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=7, value = round((solar_df.iloc[7, 1]+solar_df.iloc[8, 1])*1000,1)) # MaxProd
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=8, value = df_other_sources_VRES_PV.iloc[7, 1]) # EnableInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=9, value = df_other_sources_VRES_PV.iloc[8, 1]) # MaxInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=11, value = df_other_sources_VRES_PV.iloc[10, 1]) # OMVarCost
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=20, value=f"V2G_{simulation_year}_{scenario}")  # Which package this belongs to
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=21, value=Datasource)  # Where the data for the entry comes fromm
            cell_10 = worksheets['VRES'].cell(row=row_start + cur_row_vres, column=10) # InvestCost
            if cell_10.value is None:
                cell_10.value = 0

            cur_row_vres += 1 # Move to the next row

        # Process data for Concentrated Solar Power (CSP)
        if round(solar_df.iloc[6, 1]+solar_df.iloc[9, 1],1) > 0:
            # Add data for Concentrated Solar Power to the worksheet
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=3, value = filename.split('_')[1]+'CSP') # PP Name
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=4, value = df_other_sources_VRES_CSP.iloc[3, 1]) # Tec
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=5, value = filename.split('_')[1]) # Node/Businfo
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=6, value = df_other_sources_VRES_CSP.iloc[5, 1]) # ExisUnit
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=7, value = round((solar_df.iloc[6, 1]+solar_df.iloc[9, 1])*1000,1)) # MaxProd
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=8, value = df_other_sources_VRES_CSP.iloc[7, 1]) # EnableInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=9, value = df_other_sources_VRES_CSP.iloc[8, 1]) # MaxInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=11, value = df_other_sources_VRES_CSP.iloc[10, 1]) # OMVarCost
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=20, value=f"V2G_{simulation_year}_{scenario}")  # Which package this belongs to
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=21, value=Datasource)  # Where the data for the entry comes from
            cell_10 = worksheets['VRES'].cell(row=row_start + cur_row_vres, column=10) # InvestCost
            if cell_10.value is None:
                cell_10.value = 0

            cur_row_vres += 1 # Move to the next row

        # Process data for Other Renewable Energy Sources (OtherRES)
        if round(otherRes_df.iloc[7, 4]+otherRes_df.iloc[7, 5]+otherRes_df.iloc[7, 6]+otherRes_df.iloc[7, 7]+otherRes_df.iloc[7, 8],1) > 0:
             # Add data for Other Renewable Energy Sources to the worksheet
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=3, value = filename.split('_')[1]+'OTHERRES') # PP Name
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=4, value = df_other_sources_VRES_OtherRES.iloc[3, 1]) # Tec
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=5, value = filename.split('_')[1]) # Node/Businfo
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=6, value = df_other_sources_VRES_OtherRES.iloc[5, 1]) # ExisUnit
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=7, value = round(otherRes_df.iloc[7, 4]+otherRes_df.iloc[7, 5]+otherRes_df.iloc[7, 6]+otherRes_df.iloc[7, 7]+otherRes_df.iloc[7, 8],1)) # MaxProd
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=8, value = df_other_sources_VRES_OtherRES.iloc[7, 1]) # EnableInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=9, value = df_other_sources_VRES_OtherRES.iloc[8, 1]) # MaxInvest
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=11, value = df_other_sources_VRES_OtherRES.iloc[10, 1]) # OMVarCost
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=20, value=f"V2G_{simulation_year}_{scenario}")  # Which package this belongs to
            worksheets['VRES'].cell(row=row_start+cur_row_vres, column=21, value=Datasource)  # Where the data for the entry comes from
            cell_10 = worksheets['VRES'].cell(row=row_start + cur_row_vres, column=10) # InvestCost
            if cell_10.value is None:
                cell_10.value = 0

            cur_row_vres += 1 # Move to the next row

        # Mapping of thermal power sources to their respective DataFrames
        thermal_sources = {
            "NUCLEAR": df_other_sources_Thermal_Nuclear,
            "COALOLD1": df_other_sources_Thermal_CoalOld1,
            "COALOLD2": df_other_sources_Thermal_CoalOld2,
            "COALNEW": df_other_sources_Thermal_CoalNew,
            "COALCCS": df_other_sources_Thermal_CoalCCS,
            "LIGNITEOLD1": df_other_sources_Thermal_LigniteOld1,
            "LIGNITEOLD2": df_other_sources_Thermal_LigniteOld2,
            "LIGNITENEW": df_other_sources_Thermal_LigniteNew,
            "LIGNITECCS": df_other_sources_Thermal_LigniteCCS,
            "GASCONVOLD1": df_other_sources_Thermal_GasConvOld1,
            "GASCONVOLD2": df_other_sources_Thermal_GasConvOld2,
            "GASCCGTOLD1": df_other_sources_Thermal_GasCCGTOld1,
            "GASCCGTOLD2": df_other_sources_Thermal_GasCCGTOld2,
            "GASCCGTNEW": df_other_sources_Thermal_GasCCGTNew,
            "GASCCGTCCS": df_other_sources_Thermal_GasCCGTCCS,
            "GASOCGTOLD": df_other_sources_Thermal_GasOCGTOld,
            "GASOCGTNEW": df_other_sources_Thermal_GasOCGTNew,
            "LIGHTOIL": df_other_sources_Thermal_LightOil,
            "HEAVYOILOLD1": df_other_sources_Thermal_HeavyOilOld1,
            "HEAVYOILOLD2": df_other_sources_Thermal_HeavyOilOld2,
            "OILSHALEOLD": df_other_sources_Thermal_OilShaleOld,
            "OILSHALENEW": df_other_sources_Thermal_OilShaleNew,
            "GASCCGTPRESENT1": df_other_sources_Thermal_GasCCGTPresent1,
            "GASCCGTPRESENT2": df_other_sources_Thermal_GasCCGTPresent2,
        }

        # Define fuel price index for each thermal power source
        price_indices = {
            "NUCLEAR": 2, "COALOLD1": 7, "COALOLD2": 7, "COALNEW": 7, "COALCCS": 7, "LIGNITEOLD1": 3, "LIGNITEOLD2": 3, "LIGNITENEW": 3, "LIGNITECCS": 3,
            "GASCONVOLD1": 8, "GASCONVOLD2": 8, "GASCCGTOLD1": 8, "GASCCGTOLD2": 8, "GASCCGTPRESENT1": 8,
            "GASCCGTPRESENT2": 8, "GASCCGTNEW": 8, "GASCCGTCCS": 8, "GASOCGTOLD": 8, "GASOCGTNEW": 8,
            "LIGHTOIL": 14, "HEAVYOILOLD1": 15, "HEAVYOILOLD2": 15, "OILSHALEOLD": 20, "OILSHALENEW": 20,
        }

        # Iterate over each thermal power source and write data to the worksheet
        for idx, (source_name, df_source) in enumerate(thermal_sources.items(), start=10):
            actual_idx = 10 + (idx - 10) * 2  # Ensure idx increments by 2 instead of 1

            # Process only if MaxProd is greater than 0
            if round(thermal_df.iloc[actual_idx, 2], 1) > 0:
                price_index = price_indices.get(source_name)  # Get fuel price index

                # Extract country code from filename
                filename_part = filename.split('_')[1]
                if filename_part[2:4] == '00':
                    countryCode = filename_part[:2]  # First two characters
                else:
                    countryCode = filename_part  # Full identifier
                row_start = 8
                # Write data to the 'Thermal' worksheet
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=3, value=filename.split('_')[1] + source_name)  # PP Name
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=4, value=df_source.iloc[3, 1])  # Tec
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=5, value=filename.split('_')[1])  # Node/Businfo
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=6, value=df_source.iloc[5, 1])  # ExisUnit
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=7, value=round(thermal_df.iloc[actual_idx, 2], 1))  # MaxProd
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=8, value=df_source.iloc[7, 1])  # MinProd
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=9, value=round(thermal_df.iloc[actual_idx, 2], 1))  # RampUp
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=10, value=round(thermal_df.iloc[actual_idx, 2], 1))  # RampDw
                
                # Determine fuel cost based on country and source type
                if ("lignite" in source_name.lower() and countryCode in ['BG', 'MK', 'CZ']):
                    worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=16, value=round(df_prices.iloc[3, 2] * 3.6 , 4))
                elif ("lignite" in source_name.lower() and countryCode in ['SK', 'DE', 'RS', 'PL', 'ME', 'UKNI', 'BA', 'IE']):
                    worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=16, value=round(df_prices.iloc[4, 2] * 3.6 , 4)) # FuelCost from €/GJ in €/MWh
                elif ("lignite" in source_name.lower() and countryCode in ['SI', 'RO', 'HU']):
                    worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=16, value=round(df_prices.iloc[5, 2] * 3.6 , 4))
                elif ("lignite" in source_name.lower() and countryCode in ['GR', 'TR']):
                    worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=16, value=round(df_prices.iloc[6, 2] * 3.6 , 4))
                elif ("lignite" in source_name.lower()):
                    worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=16, value=99999)
                else:
                    worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=16, value=round(df_prices.iloc[price_index, 2] * 3.6 , 4))  # FuelCost
                
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=17, value=round(df_source.iloc[14, 1], 4))  # Efficiency
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=18, value=round(df_source.iloc[15, 1]/ 860.42065,4))  # Consumption in MWh/h
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=19, value=df_source.iloc[16, 1])  # OMVarCost
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=20, value=round(df_source.iloc[17, 1]/ 860.42065,4))  # StartUpCost in MW
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=22, value=df_source.iloc[19, 1])  # EnableInvest
                worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=25, value=round(df_source.iloc[22, 1] * 3.6, 3))  # CO2Emis
                worksheets['Thermal'].cell(row=row_start+cur_row_vres, column=30, value=f"V2G_{simulation_year}_{scenario}")  # Which package this belongs to
                worksheets['Thermal'].cell(row=row_start+cur_row_vres, column=31, value=Datasource)  # Where the data for the entry comes from
                
                cell_11 = worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=11) # MinUpTime
                if cell_11.value is None:
                    cell_11.value = 0
                cell_12 = worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=12) # MinDownTime
                if cell_12.value is None:
                    cell_12.value = 0
                cell_17 = worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=17) # Efficiency
                if cell_17.value is None:
                    cell_17.value = 1
                cell_23 = worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=23) # InvestmentCost
                if cell_23.value is None:
                    cell_23.value = 0
                cur_row_thermal += 1  # Move to the next row

        # Save workbook after processing all thermal power sources
        # Check if the file is open before saving
        if is_file_open(files["Thermal"][1]):
            sys.exit(f"File '{files['Thermal'][1]}' is currently open by another application. Please close it and try again.")
        workbooks["Thermal"].save(files["Thermal"][1])

        if round(otherNonRes_df.iloc[7, 2],1) > 0:
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=3, value=filename.split('_')[1] + 'OTHERNONRES')  # PP Name
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=4, value=df_other_sources_Thermal_OtherNonRES.iloc[3, 1])  # Tec
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=5, value=filename.split('_')[1])  # Node/Businfo
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=6, value=df_other_sources_Thermal_OtherNonRES.iloc[5, 1])  # ExisUnit
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=7, value=round(otherNonRes_df.iloc[7, 2], 1))  # MaxProd
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=8, value=df_other_sources_Thermal_OtherNonRES.iloc[7, 1])  # MinProd
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=9, value=round(otherNonRes_df.iloc[7, 2], 1))  # RampUp
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=10, value=round(otherNonRes_df.iloc[7, 2], 1))  # RampDw
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=16, value=round(df_prices.iloc[12, 2] * 3.6 , 4))  # FuelCost from €/GJ in €/MWh
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=17, value=round( df_other_sources_Thermal_OtherNonRES.iloc[14, 1], 4))  # Efficiency
            cell_17 = worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=17)
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=18, value=round(df_other_sources_Thermal_OtherNonRES.iloc[15, 1]/ 860.42065,4))  # Consumption in MWh/h
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=19, value=df_other_sources_Thermal_OtherNonRES.iloc[16, 1])  # OMVarCost
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=20, value=round(df_other_sources_Thermal_OtherNonRES.iloc[17, 1]/ 860.42065,4))  # StartUpCost
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=22, value=df_other_sources_Thermal_OtherNonRES.iloc[19, 1])  # EnableInvest
            worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=25, value=round(df_other_sources_Thermal_OtherNonRES.iloc[22, 1] * 3.6, 3))  # CO2Emis
            worksheets['Thermal'].cell(row=row_start+cur_row_vres, column=30, value=f"V2G_{simulation_year}_{scenario}")  # Which package this belongs to
            worksheets['Thermal'].cell(row=row_start+cur_row_vres, column=31, value=Datasource)  # Where the data for the entry comes from
            # Set 0 to columns if empty since the Pyomo model requires values in these columns
            cell_11 = worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=11) # MinUpTime
            if cell_11.value is None:
                cell_11.value = 0
            cell_12 = worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=12) # MinDownTime
            if cell_12.value is None:
                cell_12.value = 0
            cell_17 = worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=17) # Efficiency
            if cell_17.value is None:
                cell_17.value = 1
            cell_23 = worksheets['Thermal'].cell(row=row_start + cur_row_thermal, column=23) # InvestmentCost
            if cell_23.value is None:
                cell_23.value = 0

            cur_row_thermal += 1 # Move to the next row

# Save and close notebooks
workbooks["RoR"].save(files["RoR"][1])
workbooks["RoR"].close()
workbooks["Storage"].save(files["Storage"][1])
workbooks["Storage"].close()
workbooks["VRES"].save(files["VRES"][1])
workbooks["VRES"].close()
workbooks["Thermal"].save(files["Thermal"][1])
workbooks["Thermal"].close()

print(f"Generator data saved successfully copied.")


### 3.8 Power_VRESProfiles

In [ ]:
# Define file paths for source and destination files
if simulation_year == 2030:
    folder_source_files_VRESProfiles = os.path.join(tyndp_dir, 'PECD/2030')
    folder_source_files_OtherRESProfiles = os.path.join(tyndp_dir, 'PEMMDB2/2030')
elif simulation_year == 2040:
    folder_source_files_VRESProfiles = os.path.join(tyndp_dir, 'PECD/2040')
    folder_source_files_OtherRESProfiles = os.path.join(tyndp_dir, 'PEMMDB2/2040')
elif simulation_year == 2050:
    folder_source_files_VRESProfiles = os.path.join(tyndp_dir, 'PECD/2050')
    folder_source_files_OtherRESProfiles = os.path.join(tyndp_dir, 'PEMMDB2/2050')
else:
    sys.exit(f"Invalid simulation year: {simulation_year}. Please use 2030, 2040, or 2050.")

template_file_VRESProfiles = os.path.join(template_dir, 'Power_VRESProfiles.xlsx')
destination_file_VRESProfiles = os.path.join(destination_dir, 'Power_VRESProfiles.xlsx')

# Prepare the destination file by removing it if it already exists
if os.path.exists(destination_file_VRESProfiles):
    if is_file_open(destination_file_VRESProfiles):
        sys.exit(f"File '{destination_file_VRESProfiles}' is currently open by another application.")
    os.remove(destination_file_VRESProfiles)

# Copy the template file to the destination directory
shutil.copy(template_file_VRESProfiles, destination_file_VRESProfiles)

# If there is no file for Wind_Offshore_2030_BEOF, copy the Wind_Offshore_2030_BE00 file and rename it
if not os.path.exists(os.path.join(tyndp_dir, 'PECD/2030/PECD_Wind_Offshore_2030_BEOF_edition 2023.2.csv')):
    shutil.copy(os.path.join(tyndp_dir, 'PECD/2030/PECD_Wind_Offshore_2030_BE00_edition 2023.2.csv'), os.path.join(tyndp_dir, 'PECD/2030/PECD_Wind_Offshore_2030_BEOF_edition 2023.2.csv'))

# Load the destination workbook and worksheet
destination_wb = load_workbook(destination_file_VRESProfiles)
destination_ws = destination_wb.active

# Get the list of VRES profile files to process
vres_profiles_files = sorted(f for f in os.listdir(folder_source_files_VRESProfiles) if (f.startswith("PECD") and f.endswith(".csv")))

# Starting row in destination file
row_start = 8

# Process each VRES profile file
for filename in vres_profiles_files:
    file_path = os.path.join(folder_source_files_VRESProfiles, filename)  # Full file path
    if os.path.isfile(file_path):  # Ensure it's a valid file

        # Identify the type of technology from the filename
        matched_tec = [tec for tec in ['_CSP_noStorage_', '_LFSolarPV_', '_LFSolarPVRooftop_', '_Wind_Offshore_', '_Wind_Onshore_'] if tec in filename]
        if matched_tec:
            print(file_path) # Log the file being processed
        else:
            continue # Skip files that don't match the expected technologies
            
        # Load the VRES profile data (skip header rows, select the first 8760 rows for hourly data)
        vres_profiles = pd.read_csv(file_path, skiprows=10, header=0, nrows=8760)

        # Check if the selected year is present in the columns (year columns start from index 2)
        year_columns = vres_profiles.columns[2:]
        for y in year_columns:
            if '.' in y:
                year_columns = year_columns.str.replace('.0', '') # Remove .0 suffix if present
        if (str(cy_year) not in year_columns):
            print(f"Year {cy_year} not found in sheet {filename}. Skipping node.")
            continue # Skip if the year is not found

        # Get the column index for the selected year
        year_col_idx = year_columns.get_loc(str(cy_year)) + 2  # Adjust for 1-based index in Excel


        # Extract the time series for the selected year (8760 hours)
        time_series = vres_profiles.iloc[:, year_col_idx].fillna(0) # Fill missing values with 0

        # Skip if the entire time series is zero
        if time_series.sum() == 0:
            continue
        
        # Write data to the destination worksheet (PP name, node information)
        destination_ws[f'C{row_start}'] = 'rp01' # Periode
        destination_ws[f'D{row_start}'] = filename.split('_')[-2]  # Write node name (second last part of the filename)
        if filename.split('_')[1] + filename.split('_')[2]  == 'CSPnoStorage':
            destination_ws[f'D{row_start}'] = (destination_ws[f'D{row_start}'].value or '') + 'CSP'
        elif filename.split('_')[1] in ['LFSolarPV', 'LFSolarPVRooftop']:
            destination_ws[f'D{row_start}'] = (destination_ws[f'D{row_start}'].value or '') + 'PV'
        elif filename.split('_')[1]  + filename.split('_')[2] == 'WindOffshore':
            destination_ws[f'D{row_start}'] = (destination_ws[f'D{row_start}'].value or '') + 'WINDOFF'
        elif filename.split('_')[1] + filename.split('_')[2] == 'WindOnshore':
            destination_ws[f'D{row_start}'] = (destination_ws[f'D{row_start}'].value or '') + 'WIND'
        else:
            continue  # Skip if the technology is not recognized
        destination_ws[f'E{row_start}'] = f"V2G_{simulation_year}_{scenario}" # Which package this belongs to
        destination_ws[f'F{row_start}'] = Datasource # Where the data for the entry comes from

        # Write the time series data (8760 hours) starting from column G
        column_start = 7
        for cf in time_series: # Iterate over the time series values
            destination_ws.cell(row=row_start, column=column_start, value=round(cf,4))
            column_start += 1 # Move to the next column
    
        row_start += 1  # Move to the next row for the next entry

# Process Other RES profile files
otherRES_profiles_files = sorted(f for f in os.listdir(folder_source_files_OtherRESProfiles) if (f.startswith("PEMMDB") and f.endswith(".xlsx")))

for filename in otherRES_profiles_files:
    file_path = os.path.join(folder_source_files_OtherRESProfiles, filename)  # Full file path
    if os.path.isfile(file_path):  # Ensure it's a valid file

        # Load the Excel file and process the data
        generators_wb = pd.ExcelFile(file_path)
        print(file_path) # Log the file being processed

        otherRes_df = generators_wb.parse('Other RES')

        # Check if the total generation for the selected year is greater than zero
        if round(otherRes_df.iloc[7, 4]+otherRes_df.iloc[7, 5]+otherRes_df.iloc[7, 6]+otherRes_df.iloc[7, 7]+otherRes_df.iloc[7, 8],1) > 0:
            destination_ws[f'C{row_start}'] = 'rp01' # Periode
            destination_ws[f'D{row_start}'] = filename.split('_')[1] +'OTHERRES' # Node name
            destination_ws[f'E{row_start}'] = f"V2G_{simulation_year}_{scenario}" # Which package this belongs to
            destination_ws[f'F{row_start}'] = Datasource # Where the data for the entry comes from
            # Write the time series data (8760 hours) starting from column G
            column_start = 7
            for i in range(8760): # Iterate over the time series values
                destination_ws.cell(row=row_start, column=column_start, value=round((otherRes_df.iloc[9, 4]+otherRes_df.iloc[9, 5]+otherRes_df.iloc[9, 6]+otherRes_df.iloc[9, 7]+otherRes_df.iloc[9, 8])/(otherRes_df.iloc[7, 4]+otherRes_df.iloc[7, 5]+otherRes_df.iloc[7, 6]+otherRes_df.iloc[7, 7]+otherRes_df.iloc[7, 8]),4))
                column_start += 1 # Move to the next column
        
            row_start += 1  # Move to the next row for the next entry

# Save and close the modified workbook to the destination
destination_wb.save(destination_file_VRESProfiles)
destination_wb.close()

print(f"Data saved successfully at: {destination_file_VRESProfiles}")

---
## 4. Post-Tasks
This includes extracting or generating electrical parameters for power lines from PyPSA data and removing unnecessary data.

### 4.1 Electric Parameters for Network
The electrical parameters (X values) for power lines are sourced from PyPSA data stored in PyPSA_Data/Power_Network_PyPSA.xlsx.
If a connection is not available in the PyPSA data, a rough estimate is derived based on the relationship between X and Pmax within the existing PyPSA dataset.

In [122]:
# Define file paths for source and destination files
source_file_line_parameters = os.path.join(pypsa_dir, 'Power_Network_PyPSA.xlsx')
destination_file_Network = os.path.join(destination_dir, 'Power_Network.xlsx')

if is_file_open(destination_file_Network):
    sys.exit(f"File '{destination_file_Network}' is currently open by another application.")

# Load data from the source and destination Excel files
df_pypsa_lines = pd.read_excel(source_file_line_parameters, skiprows=6) # Data from PyPSA source file
#print(df_pypsa_lines.head()) # Print the first few rows of the PyPSA line data
df_power_network = pd.read_excel(destination_file_Network, skiprows=6) # Data from the destination network file
#print(df_power_network.head()) # Print the first few rows of the PyPSA line data
# Create a combined identifier by concatenating columns 1 and 2, removing spaces
df_pypsa_lines["combined"] = (df_pypsa_lines.iloc[:, 1] + df_pypsa_lines.iloc[:, 2]).str.replace(" ", "", regex=True)
df_pypsa_lines = df_pypsa_lines.drop_duplicates(subset="combined") # Remove duplicate combined entries

# Create a combined identifier for the power network file, similar to the PyPSA file
df_power_network["combined"] = df_power_network["combined"] = (df_power_network.iloc[:, 1] + df_power_network.iloc[:, 2]).str.replace(" ", "", regex=True)

# Map the line parameters from the PyPSA file to the destination network file based on the "combined" column
df_power_network.iloc[:, 6] = df_power_network["combined"].map(
    df_pypsa_lines.set_index("combined").iloc[:, 6]
)
print(df_power_network.iloc[:, 6].head()) # Print the first few rows of the line data
# Fill missing values in the line parameters using a mathematical formula
df_power_network.iloc[:, 10] = pd.to_numeric(df_power_network.iloc[:, 10], errors='coerce')
print(df_power_network.iloc[:, 10].head()) # Print the first few rows of the line data
df_power_network.iloc[:, 6] = df_power_network.iloc[:, 6].fillna(
    14527 * df_power_network.iloc[:, 10] ** (-0.77)
)

# Calculate the line parameters in pu
df_power_network.iloc[:, 6] = round(df_power_network.iloc[:, 6]/(((380*10**3)**2)/(100*10**6)), 15)

# Load the destination workbook and worksheet for saving the changes
destination_wb = load_workbook(destination_file_Network)
destination_ws = destination_wb.active

# Write the adjusted line parameters into the destination worksheet, starting from row 8
start_row = 8
for i, value in enumerate(df_power_network.iloc[:, 0], start=start_row):
    destination_ws[f'G{i}'] = df_power_network.iloc[i-start_row, 6]

# Save the modified workbook to the destination
destination_wb.save(destination_file_Network)
destination_wb.close()

print("Line parameters updated successfully.")

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: [p.u.].1, dtype: float64
0     600
1     400
2     500
3     250
4    1200
Name: [MW], dtype: int64
Line parameters updated successfully.


### 4.2 Data Cleanup
Removes unnecessary data that was copied during processing:

- Deletes demand and generators from nodes that are not listed in *Power_BusInfo.xlsx*.
- Removes inflows for RoR or storage units that are not listed in *Power_RoR.xlsx* or *Power_Storage*.
- Deletes NTCs and power lines that reference buses not found in *P*ower_BusInfo.xlsx*.
- Removes VRES profiles for VRES generators units that are not listed in *Power_VRES*.

**IMPORTANT**:
If you want to include only a subset of countries in the final dataset, first remove all unwanted countries from *Power_BusInfo.xlsx* in the specified `destination_dir`.

#### 4.2.1 Power_Demand
Deletes demand from nodes that are not listed in *Power_BusInfo.xlsx*.

In [ ]:
# Define file paths for Power Bus Information and Power Demand data
file_businfo = os.path.join(destination_dir, 'Power_BusInfo.xlsx')
file_demand = os.path.join(destination_dir, 'Power_Demand.xlsx')

if is_file_open(file_demand):
    sys.exit(f"File '{file_demand}' is currently open by another application.")

# Load Power_BusInfo.xlsx and extract valid nodes into a set
bus_wb = load_workbook(file_businfo, data_only=True) # Load workbook with data-only mode
bus_ws = bus_wb.active

valid_nodes = set() # Set to store valid node IDs
for row in bus_ws.iter_rows(min_row=8, min_col=3, max_col=3, values_only=True):
    if row[0] is not None:
        valid_nodes.add(str(row[0])) # Add valid node IDs to the set

# Load Power_Demand.xlsx to filter nodes based on validity
demand_wb = load_workbook(file_demand) # Load the Power Demand workbook
demand_ws = demand_wb.active

# Collect rows to delete based on invalid node values
delete_rows = [] # List to store rows that should be deleted
for row_idx, row in enumerate(demand_ws.iter_rows(min_row=8, min_col=4, max_col=4, values_only=True), start=8):
    if row[0] is None or str(row[0]) not in valid_nodes: # Check for invalid or missing nodes
        delete_rows.append(row_idx) # Add invalid rows to the delete list
        print(f"Row {row_idx} with node {row[0]} will be deleted.")

# Delete rows in reverse order to avoid index shifting during deletion
print("Deleting rows (can take a while for large datasets)...")
for row_idx in reversed(delete_rows):
    demand_ws.delete_rows(row_idx) # Delete invalid rows from the workbook

# Save and close the updated Power_Demand.xlsx
demand_wb.save(file_demand)
demand_wb.close()

print("Power Demand data cleaned successfully.")

#### 4.2.2 Power_RoR
Deletes RoR generators with nodes that are not listed in *Power_BusInfo.xlsx*.

In [ ]:
# Define file paths for Power Bus Information and Power RoR data
file_businfo = os.path.join(destination_dir, 'Power_BusInfo.xlsx')
file_RoR = os.path.join(destination_dir, 'Power_RoR.xlsx')

if is_file_open(file_RoR):
    sys.exit(f"File '{file_RoR}' is currently open by another application.")

# Load Power_BusInfo.xlsx to extract valid nodes into a set
bus_wb = load_workbook(file_businfo, data_only=True) # Load workbook with data-only mode
bus_ws = bus_wb.active

valid_nodes = set() # Set to store valid node IDs
for row in bus_ws.iter_rows(min_row=8, min_col=3, max_col=3, values_only=True):
    if row[0] is not None:
        valid_nodes.add(str(row[0])) # Add valid node IDs to the set
# Load Power_RoR.xlsx to filter nodes based on validity
ror_wb = load_workbook(file_RoR) # Load the Power RoR workbook
ror_ws = ror_wb.active

# Identify rows to delete based on invalid or missing nodes
delete_rows = [] # List to store rows that need to be deleted
for row_idx, row in enumerate(ror_ws.iter_rows(min_row=7, min_col=4, max_col=4, values_only=True), start=7):
    node = str(row[0]).strip().upper() if row[0] is not None else None
    if node is None or node not in valid_nodes:
        delete_rows.append(row_idx) # Add invalid rows to the delete list
        print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion of the row

# Delete rows in reverse order to prevent shifting issues during deletion
print("Deleting rows (can take a while for large datasets)...")
for row_idx in reversed(delete_rows):
    ror_ws.delete_rows(row_idx) # Delete the invalid rows

# Save and close the filtered Power_RoR.xlsx with updated data
ror_wb.save(file_RoR)
ror_wb.close()

print("Power RoR data cleaned successfully.")

#### 4.2.3 Power_Storage
Deletes storage units with nodes that are not listed in *Power_BusInfo.xlsx*.

In [ ]:
# Define file paths for Power Bus Information and Power Storage data
file_businfo = os.path.join(destination_dir, 'Power_BusInfo.xlsx')
file_storage = os.path.join(destination_dir, 'Power_Storage.xlsx')

if is_file_open(file_storage):
    sys.exit(f"File '{file_storage}' is currently open by another application.")

# Load Power_BusInfo.xlsx to extract valid nodes into a set
bus_wb = load_workbook(file_businfo, data_only=True) # Load workbook with data-only mode
bus_ws = bus_wb.active

valid_nodes = set() # Set to store valid node IDs
for row in bus_ws.iter_rows(min_row=8, min_col=3, max_col=3, values_only=True):
    if row[0] is not None:
        valid_nodes.add(str(row[0])) # Add valid node IDs to the set

# Load Power_RoR.xlsx to filter nodes based on validity
storage_wb = load_workbook(file_storage)
storage_ws = storage_wb.active

# Identify rows to delete based on invalid or missing nodes
delete_rows = [] # List to store rows that need to be deleted
for row_idx, row in enumerate(storage_ws.iter_rows(min_row=7, min_col=4, max_col=4, values_only=True), start=7):
    node = str(row[0]).strip().upper() if row[0] is not None else None
    if node is None or node not in valid_nodes:
        delete_rows.append(row_idx) # Add invalid rows to the delete list
        print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion of the row

# Delete rows in reverse order to prevent shifting issues during deletion
print("Deleting rows (can take a while for large datasets)...")
for row_idx in reversed(delete_rows):
    storage_ws.delete_rows(row_idx) # Delete the invalid rows

# Save and close the filtered Power_Storage.xlsx with updated data
storage_wb.save(file_storage)
storage_wb.close()

print("Power Storage data cleaned successfully.")

#### 4.2.4 Power_VRES
Deletes VRES generators with nodes that are not listed in *Power_BusInfo.xlsx*.

In [ ]:
# Define file paths for Power Bus Information and Power VRES data
file_businfo = os.path.join(destination_dir, 'Power_BusInfo.xlsx')
file_VRES = os.path.join(destination_dir, 'Power_VRES.xlsx')

if is_file_open(file_VRES):
    sys.exit(f"File '{file_VRES}' is currently open by another application.")

# Load Power_BusInfo.xlsx to extract valid nodes into a set
bus_wb = load_workbook(file_businfo, data_only=True) # Load workbook with data-only mode
bus_ws = bus_wb.active

valid_nodes = set() # Set to store valid node IDs
for row in bus_ws.iter_rows(min_row=8, min_col=3, max_col=3, values_only=True):
    if row[0] is not None:
        valid_nodes.add(str(row[0])) # Add valid node IDs to the set
# Load Power_VRES.xlsx to filter nodes based on validity
vres_wb = load_workbook(file_VRES) # Load the Power VRES workbook
vres_ws = vres_wb.active

# Identify rows to delete based on invalid or missing nodes
delete_rows = [] # List to store rows that need to be deleted
for row_idx, row in enumerate(vres_ws.iter_rows(min_row=8, min_col=5, max_col=5, values_only=True), start=8):
    node = str(row[0]).strip().upper() if row[0] is not None else None
    if node is None or node not in valid_nodes:
        delete_rows.append(row_idx) # Add invalid rows to the delete list
        print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion of the row

# Delete rows in reverse order to prevent shifting issues during deletion
print("Deleting rows (can take a while for large datasets)...")
for row_idx in reversed(delete_rows):
    vres_ws.delete_rows(row_idx) # Delete the invalid rows
    
vres_wb.save(file_VRES)
vres_wb.close()

print("Power VRES data cleaned successfully.")

#### 4.2.5 Power_ThermalGen
Deletes thermal generators with nodes that are not listed in *Power_BusInfo.xlsx*.

In [ ]:
# Define file paths for Power Bus Information and Power ThermalGen data
ile_businfo = os.path.join(destination_dir, 'Power_BusInfo.xlsx')
file_thermals = os.path.join(destination_dir, 'Power_ThermalGen.xlsx')

if is_file_open(file_thermals):
    sys.exit(f"File '{file_thermals}' is currently open by another application.")

# Load Power_BusInfo.xlsx to extract valid nodes into a set
bus_wb = load_workbook(file_businfo, data_only=True) # Load workbook with data-only mode
bus_ws = bus_wb.active

valid_nodes = set() # Set to store valid node IDs
for row in bus_ws.iter_rows(min_row=8, min_col=3, max_col=3, values_only=True):
    if row[0] is not None:
        valid_nodes.add(str(row[0])) # Add valid node IDs to the set


# Load Power_ThermalGen.xlsx to filter nodes based on validity
thermals_wb = load_workbook(file_thermals) # Load the Power ThermalGen workbook
thermals_ws = thermals_wb.active

# Identify rows to delete based on invalid or missing nodes
delete_rows = [] # List to store rows that need to be deleted
for row_idx, row in enumerate(thermals_ws.iter_rows(min_row=8, min_col=5, max_col=5, values_only=True), start=8):
    node = str(row[0]).strip().upper() if row[0] is not None else None
    if node is None or node not in valid_nodes:
        delete_rows.append(row_idx) # Add invalid rows to the delete list
        print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion of the row

# Delete rows in reverse order to prevent shifting issues during deletion
print("Deleting rows (can take a while for large datasets)...")
for row_idx in reversed(delete_rows):
    thermals_ws.delete_rows(row_idx) # Delete the invalid rows

# Identify and remove duplicate rows based on columns 4 (D) and 5 (E)
seen_pairs = set()
duplicate_rows = []

for row_idx, row in enumerate(thermals_ws.iter_rows(min_row=8, min_col=3, max_col=4, values_only=True), start=8):
    pair = (row[0], row[1])
    if pair in seen_pairs:
        duplicate_rows.append(row_idx)
        print(f"Row {row_idx} with duplicate pair {pair} marked for deletion.")
    else:
        seen_pairs.add(pair)

# Delete duplicate rows in reverse to avoid row shift
print("Deleting duplicate rows...")
for row_idx in reversed(duplicate_rows):
    thermals_ws.delete_rows(row_idx)

# Save and close the filtered Power_ThermalGen.xlsx with updated data
thermals_wb.save(file_thermals)
thermals_wb.close()

print("Power ThermalGen data cleaned successfully.")

#### 4.2.6 Power_Inflows
Removes inflows for RoR and storage units that are not listed in *Power_RoR.xlsx* or *Power_Storage*.

In [ ]:
# Define file paths for Power RoR, Power Storage, and Power Inflows
file_ror = os.path.join(destination_dir, 'Power_RoR.xlsx')
file_storage = os.path.join(destination_dir, 'Power_Storage.xlsx')
file_inflows = os.path.join(destination_dir, 'Power_Inflows.xlsx')

if is_file_open(file_inflows):
    sys.exit(f"File '{file_inflows}' is currently open by another application.")

# Load Power_RoR.xlsx to extract valid inflow nodes
ror_wb = load_workbook(file_ror, data_only=True)
ror_ws = ror_wb.active

# Load Power_Storage.xlsx to extract valid storage nodes
storage_wb = load_workbook(file_storage, data_only=True)
storage_ws = storage_wb.active

valid_inflows = set() # Set to store valid inflow node IDs

# Extract valid nodes from Power_RoR and Power_Storage workbooks
for row in ror_ws.iter_rows(min_row=7, min_col=2, max_col=2, values_only=True):
    if row[0] is not None:
        valid_inflows.add(str(row[0])) # Add valid inflows to the set

for row in storage_ws.iter_rows(min_row=7, min_col=2, max_col=2, values_only=True):
    if row[0] is not None:
        valid_inflows.add(str(row[0])) # Add valid storage nodes to the set

# Load Power_Inflows.xlsx to filter rows based on valid inflow nodes
inflows_wb = load_workbook(file_inflows) # Load the Power Inflows workbook
inflows_ws = inflows_wb.active

# Identify rows to delete based on invalid or missing inflow nodes
delete_rows = [] # List to store rows that need to be deleted
for row_idx, row in enumerate(inflows_ws.iter_rows(min_row=7, min_col=3, max_col=3, values_only=True), start=7):
    if row[0] is None or str(row[0]) not in valid_inflows: # Check if the inflow node is invalid or missing
        delete_rows.append(row_idx) # Add invalid rows to the delete list
        print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion of the row

# Delete rows in reverse order to prevent shifting issues during deletion
print("Deleting rows (can take a while for large datasets)...")
for row_idx in reversed(delete_rows):
    inflows_ws.delete_rows(row_idx) # Delete the invalid rows

# Save and close the filtered Power_Inflows.xlsx with updated data
inflows_wb.save(file_inflows)
inflows_wb.close()

print("Power Inflows data cleaned successfully.")

#### 4.2.7 Power_NTC
Deletes NTCs that reference buses not found in *P*ower_BusInfo.xlsx*.

In [ ]:
# # Define file paths for Power BusInfo and Power NTC data
# file_businfo = os.path.join(destination_dir, 'Power_BusInfo.xlsx')
# file_NTC = os.path.join(destination_dir, 'Power_NTC.xlsx')

# if is_file_open(file_NTC):
#     sys.exit(f"File '{file_NTC}' is currently open by another application.")

# # Load Power_BusInfo.xlsx to extract valid node IDs
# bus_wb = load_workbook(file_businfo, data_only=True) # Load workbook in data-only mode
# bus_ws = bus_wb.active

# valid_nodes = set() # Set to store valid node IDs

# # Extract valid nodes from Power_BusInfo
# for row in bus_ws.iter_rows(min_row=8, min_col=4, max_col=4, values_only=True):
#     if row[0] is not None:
#         valid_nodes.add(str(row[0]))

# # Load Power_NTC.xlsx to filter rows based on valid nodes
# ntc_wb = load_workbook(file_NTC) # Load the Power NTC workbook
# ntc_ws = ntc_wb.active

# # Identify rows to delete based on invalid or missing node IDs (column 2)
# delete_rows = [] # List to store rows that need to be deleted
# for row_idx, row in enumerate(ntc_ws.iter_rows(min_row=7, min_col=2, max_col=2, values_only=True), start=7):
#     if row[0] is None or str(row[0]) not in valid_nodes: # Check if the node is invalid or missing
#         delete_rows.append(row_idx) # Add row index to the delete list
#         print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion

# # Identify rows to delete based on invalid or missing node IDs (column 3)
# for row_idx, row in enumerate(ntc_ws.iter_rows(min_row=7, min_col=3, max_col=3, values_only=True), start=7):
#     if row[0] is None or str(row[0]) not in valid_nodes: # Check if the node is invalid or missing
#         delete_rows.append(row_idx) # Add row index to the delete list
#         print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion

# # Remove duplicate row indices and sort in descending order to avoid shifting issues during deletion
# delete_rows = sorted(set(delete_rows))

# # Delete rows in reverse order to prevent issues with shifting after deletion
# print("Deleting rows (can take a while for large datasets)...")
# for row_idx in reversed(delete_rows):
#     ntc_ws.delete_rows(row_idx) # Delete the rows

# # Save and close the updated Power_NTC.xlsx with the filtered data
# ntc_wb.save(file_NTC)
# ntc_wb.close()

# print("Power NTC data cleaned successfully.")

#### 4.2.8 Power_Network
Deletes lines that reference buses not found in *P*ower_BusInfo.xlsx*.

In [ ]:
# Define file paths for Power BusInfo and Power Network data
file_businfo = os.path.join(destination_dir, 'Power_BusInfo.xlsx')
file_Network = os.path.join(destination_dir, 'Power_Network.xlsx')

if is_file_open(file_Network):
    sys.exit(f"File '{file_Network}' is currently open by another application.")

# Load Power_BusInfo.xlsx to extract valid node IDs
bus_wb = load_workbook(file_businfo, data_only=True) # Load workbook in data-only mode
bus_ws = bus_wb.active

valid_nodes = set() # Set to store valid node IDs

# Extract valid nodes from Power_BusInfo
for row in bus_ws.iter_rows(min_row=8, min_col=3, max_col=3, values_only=True):
    if row[0] is not None:
        valid_nodes.add(str(row[0])) # Add valid node IDs to the set

# Load Power_Network.xlsx to filter rows based on valid nodes
network_wb = load_workbook(file_Network) # Load the Power Network workbook
network_ws = network_wb.active

# Identify rows to delete based on invalid or missing node IDs (column 2)
delete_rows = [] # List to store rows that need to be deleted
for row_idx, row in enumerate(network_ws.iter_rows(min_row=8, min_col=3, max_col=3, values_only=True), start=8):
    node = str(row[0]).strip().upper() if row[0] is not None else None
    if node is None or node not in valid_nodes:
        delete_rows.append(row_idx) # Add row index to the delete list
        print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion

# Identify second rows to delete based on invalid or missing node IDs (column 3)
for row_idx, row in enumerate(network_ws.iter_rows(min_row=8, min_col=4, max_col=4, values_only=True), start=8):
    node = str(row[0]).strip().upper() if row[0] is not None else None
    if node is None or node not in valid_nodes:
        delete_rows.append(row_idx) # Add row index to the delete list
        print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion

# Remove duplicate row indices and sort in descending order to avoid shifting issues during deletion
delete_rows = sorted(set(delete_rows))

# Delete rows in reverse order to prevent issues with shifting after deletion
print("Deleting rows (can take a while for large datasets)...")
for row_idx in reversed(delete_rows):
    network_ws.delete_rows(row_idx) # Delete the rows

# Save the updated Power_Network.xlsx with the filtered data
network_wb.save(file_Network)
network_wb.close()

print("Power Network data cleaned successfully.")

#### 4.2.9 Power_VRESProfiles
Removes VRES profiles for VRES generators units that are not listed in *Power_VRES*.

In [ ]:
# Define file paths for Power VRES and Power VRES Profiles data
file_VRES = os.path.join(destination_dir, 'Power_VRES.xlsx')
file_VRESProfiles = os.path.join(destination_dir, 'Power_VRESProfiles.xlsx')

if is_file_open(file_VRESProfiles):
    sys.exit(f"File '{file_VRESProfiles}' is currently open by another application.")

# Load Power_VRES.xlsx and extract valid node identifiers
vres_wb = load_workbook(file_VRES, data_only=True) # Load workbook in data-only mode
vres_ws = vres_wb.active

valid_nodes = set()# Set to store valid combined node identifiers

# Extract valid profiles by combining information from column 3 Generator # Old Version doen't work in new Power_VRESProfiles
for row in vres_ws.iter_rows(min_row=8, min_col=3, max_col=3, values_only=True):
        valid_nodes.add(str(row[0]) )# Add valid node IDs to the set

# Load Power_VRESProfiles.xlsx to filter rows based on valid inflows
vresprofiles_wb = load_workbook(file_VRESProfiles) # Load the Power VRES Profiles workbook
vresprofiles_ws = vresprofiles_wb.active

# Identify rows to delete based on invalid or missing combined identifiers (columns 3 and 4)
delete_rows = [] # List to store rows that need to be deleted
for row_idx, row in enumerate(vresprofiles_ws.iter_rows(min_row=8, min_col=4, max_col=4, values_only=True), start=8):
    # Combine values from columns 3 and 4 to check for valid inflow identifiers
    node = str(row[0]).strip().upper() if row[0] is not None else None
    if node is None or node not in valid_nodes:
        delete_rows.append(row_idx) # Add row index to the delete list
        print(f"Row {row_idx} with node {row[0]} is marked for deletion.") # Log the deletion

# Remove duplicate row indices and sort them in reverse order to avoid shifting issues during deletion
delete_rows = sorted(set(delete_rows))

# Delete rows in reverse order to prevent issues with shifting after deletion
print("Deleting rows (can take a while for large datasets)...")
for row_idx in reversed(delete_rows):
    vresprofiles_ws.delete_rows(row_idx) # Delete the rows

# Save and close the updated Power_VRESProfiles.xlsx with the filtered data
vresprofiles_wb.save(file_VRESProfiles)
vresprofiles_wb.close()

print("Power VRES Profiles data cleaned successfully.")

### 4.3 Copy remaining files
Copies all necessary remaining files required by LEGO that do not need modifications.

In [ ]:
# Define source and destination folder paths for file transfer
source_folder = template_dir
destination_folder = destination_dir

# Loop through all files in the source folder
for filename in os.listdir(source_folder):
    # Ignore temp Excel files (e.g., ~$filename.xlsx)
    if filename.startswith("~$"):
        print(f"Skipped temp file: {filename}")
        continue

    # Clean filename just in case (strip whitespace)
    filename_clean = filename.strip()

    # Define full file paths
    src_file = os.path.join(source_folder, filename_clean)
    dest_file = os.path.join(destination_folder, filename_clean)

    # Check if the source is a real file and the destination does not exist
    if os.path.isfile(src_file) and not os.path.exists(dest_file):
        shutil.copy2(src_file, dest_file)
        print(f"Copied: {filename_clean}")
    else:
        print(f"Skipped: {filename_clean} (already exists or not a file)")